#### Start

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os
# Load token from file
with open('/content/drive/MyDrive/agri-llm/hf_token.txt', 'r') as file:
    hf_token = file.read().strip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [35]:
!pip install python-dotenv
!pip install -q -U langchain huggingface_hub sentence_transformers faiss-gpu
!pip install -U langchain-community
!pip install transformers datasets torch scikit-learn numpy

In [69]:
os.chdir('/content/drive/MyDrive/agri-llm')
print(os.getcwd())
from models.rag.config_setup import *
from models.rag.fn_rag import *
from evaluate.fn_ragas import *

from models.rag import config_setup as the_actual_config_setup

/content/drive/MyDrive/agri-llm


#### RAGAS Main

In [70]:
evaluation_questions = {
    "What are the primary uses of maize in industry?":
        "Maize is used for food, animal feed, and as a raw material in various industries like starch and textile.",
    "What is the optimal time for chickpea planting for  peninsular  India?":
        "For  peninsular  India,  first  fort night of  October  is the  best  time  for  chickpea planting."
}

# Initialize your RAG system
rag = RAGSystem(
    documents_path=str(the_actual_config_setup.Config.DATA_DIR),
)
rag.load_documents()
rag.create_vector_store()
rag.setup_qa_chain()

# Initialize evaluator
evaluator = RAGEvaluator(rag_system=rag, evaluation_questions=evaluation_questions)

# Run evaluation and print results
eval_results = evaluator.evaluate()
print(format_evaluation_results(eval_results))

<class 'models.rag.fn_rag.RAGSystem'>
<class 'config_setup.Config'>
/content/drive/MyDrive/data_argi_llm/kvk_pop/LAKSHADWEEP/lakshadweep/lakshadweep
Split documents into 28 chunks


/content/drive/MyDrive/agri-llm/models/rag/fn_rag.py:71: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
/content/drive/MyDrive/agri-llm/models/rag/fn_rag.py:92: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})



Question: What are the primary uses of maize in industry?
Expected Answer: Maize is used for food, animal feed, and as a raw material in various industries like starch and textile.
RAG Answer: I'm not sure about that

Evaluation Scores:

Answer Correctness:
Score: -0.0039
Reasoning: Semantic similarity between generated and expected answer: -0.0039

Context Relevance:
Score: 0.3755
Reasoning: Semantic similarity between context and question: 0.3755

Faithfulness:
Score: 0.0506
Reasoning: Semantic similarity between answer and context: 0.0506

Answer Relevance:
Score: 0.1161
Reasoning: Semantic similarity between answer and question: 0.1161

Average Score: 0.1346

Sources Preview: traces across the islands. This information brochure is about cultivating 
vegetables in grow bags and the package of practices.  
 
Importance of grow bag farming  
 Overcomes the inadequate land fo...


Question: What is the optimal time for chickpea planting for  peninsular  India?
Expected Answer: For  p